In [8]:
import os
import json
import numpy as np
import openai
from openai import OpenAI

In [9]:
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

In [68]:
question = "Qual é a capital da França?"
prompt = f"Pergunta: {question}\nResposta:"
response = client.responses.create(
    model="gpt-4o",
    input=prompt
)

In [69]:
print(response.output_text)

Paris.


In [ ]:
# Carrega o conteúdo do arquivo .md
with open('dados-sac.md', 'r', encoding='utf-8') as f:
    sac_content = f.read()

# Divide o conteúdo em tópicos
sac_paragraphs = sac_content.split('# ')[1:]  # O primeiro tópico é vazio

In [192]:
print(len(sac_paragraphs))

11


In [201]:
print(sac_paragraphs[0])  # Exibe o primeiro tópico para verificação

Status do pedido: o que isso significa?
Quer entender melhor o que significa cada status de pedido? Então confira a lista com os principais:
Pedido confirmado
Recebemos o seu pedido.
Aguardando pagamento
Estamos aguardando o pagamento do seu pedido.
Pagamento aprovado
O pagamento do seu pedido foi aprovado! Agora vamos separar o seu pedido.
Preparando entrega
Estamos preparando seu pedido para entrega.
Pedido enviado
O pedido está a caminho do endereço de entrega.
Insucesso na Entrega: destinatário ausente
A transportadora tentou entregar o pedido, mas não encontrou o destinatário. Uma nova tentativa será feita nos próximos dias em horário comercial (das 08h às 18h).
Insucesso na Entrega: endereço não localizado
A transportadora tentou realizar a entrega, mas não encontrou o endereço informado. Entre em contato através dos nossos canais de atendimento para atualizar o endereço de entrega.
Insucesso na Entrega: pedido recusado
Houve a tentativa de entrega, mas o recebedor recusou. O ped

In [214]:
# Função para obter embedding de um texto
def get_embedding(text):
    response = openai.embeddings.create(
        input=text,
        model='text-embedding-3-small'
    )
    return response.data[0].embedding

# Cria embeddings para cada trecho
paragraph_embeddings = [get_embedding(p) for p in sac_paragraphs]

In [215]:
print(len(paragraph_embeddings))

11


In [233]:
question = "Como devolver uma roupa comprada no site C&A?"
embedding = get_embedding(question)
    
# Calcular similaridade com cada trecho
similarities = []
for idx, pemb in enumerate(paragraph_embeddings):
    sim = np.dot(embedding, pemb) / (
        np.linalg.norm(embedding) * np.linalg.norm(pemb)
    )
    similarities.append((sim, sac_paragraphs[idx]))

# Selecionar o trecho mais similar
similarities.sort(reverse=True, key=lambda x: x[0])
most_relevant_paragraph = similarities[0][1]

In [234]:
print(most_relevant_paragraph)

Como trocar ou devolver um pedido comprado no site ou app C&A?
Antes de começar, localize se o seu pedido é vendido e entregue pela C&A ou por uma marca parceira (Galeria C&A).
Produtos vendidos e entregues pela C&A
Roupas, acessórios e cosméticos: devem ter a sua solicitação de troca realizada em até 30 dias corridos, a contar da data do recebimento do produto. Se você deseja devolver, o prazo é de 7 dias corridos.
A melhor forma de trocar ou devolver um itens vendidos e entregues pela C&A é em qualquer loja C&A.
Se preferir, também pode solicitar a troca e/ou devolução destes produtos no site através da seção “Meus Pedidos” > "Troca e devolução", ou pelo nossos canais de atendimento. Neste caso, a troca e/ou devolução deverá ser feita em uma agência dos Correios ou retirada no endereço de entrega, caso a opção esteja disponível para o CEP.
Moda íntima, óculos, relógios e eletrônicos: devem ter a sua solicitação de troca realizada em até 7 dias corridos a contar da data do recebimento

In [235]:
prompt = f"Responda às perguntas com base no seguinte conhecimento:\n{most_relevant_paragraph}. Não responda perguntas para as quais você não possui contexto.\nPergunta: {question}\nResposta:"
response = client.responses.create(
    model='gpt-4o',
    input=prompt
)

In [236]:
print(response.output_text)

Para devolver uma roupa comprada no site C&A, siga estas etapas:

1. Verifique se o pedido foi vendido e entregue pela C&A.
2. A devolução deve ser solicitada em até 7 dias corridos a partir da data de recebimento.
3. A melhor forma é ir a qualquer loja C&A.
4. Alternativamente, acesse o site na seção “Meus Pedidos” > "Troca e devolução" ou use os canais de atendimento.
5. A devolução pode ser feita via Correios ou retirada no endereço, se disponível para o seu CEP.


In [4]:
with open('dados-produtos.json', 'r', encoding='utf-8') as f:
    product_catalog = json.load(f)

In [5]:
product_catalog

[{'productId': 2093380,
  'title': 'Calça Social Masculina Preta',
  'price': 119.99,
  'image': 'https://cea.vteximg.com.br/arquivos/ids/58721439/Foto-0.jpg?v=638536368259330000',
  'description': 'calca social masculina preta moda roupas calcas preto 38 40 42 44 46 48 50 52'},
 {'productId': 2101743,
  'title': 'Calça Masculina Esportiva Ace Básica com Vivo Contrastante  Preta',
  'price': 99.99,
  'image': 'https://cea.vteximg.com.br/arquivos/ids/59011730/Foto-0.jpg?v=638620916471170000',
  'description': 'calca masculina esportiva ace basica vivo contrastante preta moda roupas calcas preto g gg m p'},
 {'productId': 2101744,
  'title': 'Calça Masculina Esportiva Ace Básica com Vivo Contrastante Azul Marinho',
  'price': 89.99,
  'image': 'https://cea.vteximg.com.br/arquivos/ids/58973024/Foto-0.jpg?v=638606316724700000',
  'description': 'calca masculina esportiva ace basica vivo contrastante azul marinho moda roupas calcas g gg m p'},
 {'productId': 2127002,
  'title': 'Calça Leggi

In [24]:
products_list = 2093380
filtered_products = list(filter(lambda p: p['productId'] == products_list, product_catalog))

In [27]:
filtered_products

[{'productId': 2093380,
  'title': 'Calça Social Masculina Preta',
  'price': 119.99,
  'image': 'https://cea.vteximg.com.br/arquivos/ids/58721439/Foto-0.jpg?v=638536368259330000',
  'description': 'calca social masculina preta moda roupas calcas preto 38 40 42 44 46 48 50 52'}]

In [ ]:
# Carga de documentos de conhecimento
with open('anexo_sac.txt', 'r', encoding='utf-8') as f:
    sac_knowledge = f.read()

with open('anexo_prod.txt', 'r', encoding='utf-8') as f:
    product_catalog = f.read()

# Função de busca no conhecimento SAC (exemplo simples)
def sac_retrieval(query):
    # Implementar busca por relevância no conhecimento SAC aqui
    # por simplicidade, retornamos o conhecimento completo
    return sac_knowledge

# Função do agente SAC
def sac_agent(question):
    context = sac_retrieval(question)
    prompt = f"Responda às perguntas com base no seguinte conhecimento:\n{context}\nPergunta: {question}\nResposta:"
    response = openai.Completion.create(
        engine='text-davinci-003',
        prompt=prompt,
        max_tokens=150
    )
    return response.choices[0].text.strip()

# Função do agente Produtos
def product_agent(query):
    # Implementar busca no catálogo de produtos
    # Aqui, fazemos uma busca simples por palavras-chave
    if query.lower() in product_catalog.lower():
        return f"Produto encontrado: {query}"
    else:
        return "Produto não encontrado."

# Endpoints da API
@app.route('/iniciar', methods=['POST'])
def iniciar_conversa():
    session_id = request.json.get('session_id')
    # Opcional: criar registro no banco de dados
    return jsonify({"message": "Conversa iniciada", "session_id": session_id})

@app.route('/mensagem', methods=['POST'])
def receber_mensagem():
    session_id = request.json.get('session_id')
    message = request.json.get('message')
    agente = request.json.get('agente')  # 'sac' ou 'produto'

    # Armazena na história
    conn = sqlite3.connect('chat_history.db')
    c = conn.cursor()

    if agente == 'sac':
        resposta = sac_agent(message)
    elif agente == 'produto':
        resposta = product_agent(message)
    else:
        resposta = "Agente desconhecido."

    # Salvar histórico
    c.execute('INSERT INTO history (session_id, sender, message, response) VALUES (?, ?, ?, ?)',
              (session_id, 'user', message, resposta))
    conn.commit()
    conn.close()

    return jsonify({"response": resposta})

@app.route('/historico/<session_id>', methods=['GET'])
def obter_histórico(session_id):
    conn = sqlite3.connect('chat_history.db')
    c = conn.cursor()
    c.execute('SELECT sender, message, response FROM history WHERE session_id = ?', (session_id,))
    rows = c.fetchall()
    conn.close()
    history = [{"sender": r[0], "message": r[1], "response": r[2]} for r in rows]
    return jsonify(history)

if __name__ == '__main__':
    app.run(debug=True)